# Advent of Code 2021 - Day 22

In [127]:
data class Cube(val x: Int, val y: Int, val z: Int)

sealed class CuboidSet(set: Set<Cube>) : Set<Cube> by set {
    constructor(width: IntRange, height: IntRange, depth: IntRange) : this(
        width.fold(mutableSetOf<Cube>()) { outer, x ->
            height.fold(outer) { inner , y ->
                depth.fold(inner) { acc, z ->
                    acc.apply { add(Cube(x, y, z)) }
                }
            }
        }
    )

    class On : CuboidSet {
        constructor(set: Set<Cube>) : super(set)
        constructor(width: IntRange, height: IntRange, depth: IntRange) : super(width, height, depth)
    }

    class Off : CuboidSet {
        constructor(set: Set<Cube>) : super(set)
        constructor(width: IntRange, height: IntRange, depth: IntRange) : super(width, height, depth)
    }
}

In [128]:
import java.io.File

operator fun IntRange.contains(other: IntRange): Boolean = (other.start in this) && (other.endInclusive in this)

val regex = Regex("""(on|off) x=(-?\d+)..(-?\d+),y=(-?\d+)..(-?\d+),z=(-?\d+)..(-?\d+)""")

val INIT_RANGE: IntRange = (-50..50)

val (initialization: List<CuboidSet>, _: Unit) = File("Day22.input.txt")
    .bufferedReader()
    .readLines()
    .map { regex.find(it)!!.destructured }
    .map { (on, x1, x2, y1, y2, z1, z2) ->
        on to listOf(x1.toInt()..x2.toInt(), y1.toInt()..y2.toInt(), z1.toInt()..z2.toInt())
    }
    .partition { (_, dimension) ->
        dimension[0] in INIT_RANGE && dimension[1] in INIT_RANGE && dimension[2] in INIT_RANGE 
    }
    .let { (initCuboids, _) ->
        initCuboids.map { (on, dimension) ->
            if (on == "on") CuboidSet.On(dimension[0], dimension[1], dimension[2]) 
            else CuboidSet.Off(dimension[0], dimension[1], dimension[2])
        } to Unit
    }

## Part 1

Given an `initialization` list of `CuboidSets`, find the number of `Cube`s that are on.

In [129]:
fun List<CuboidSet>.onCubes(): Int = reduce { acc, cuboid ->
    when (cuboid) {
        is CuboidSet.On -> CuboidSet.On(acc union cuboid)
        is CuboidSet.Off -> CuboidSet.Off(acc subtract cuboid)
    }
}.size

initialization.onCubes()

591365

### Notes

So I feared this would be the "brute force" way to do it, and after seeing part 2, it definitely is. At least for part 1 though, the idea is to `union` all of the cuboids that are on and to `subtract` the cuboids that are off. This works pretty well for (relatively) small sets, but when the sample input says that 2758514936282235 cubes are on in `bigInitialization`, things don't look so good for this algorithm. However, we might be able to utilize the same "idea" for part 2. Part 1 is super clean though 🤷‍♂️.

## Part 2

Part 1 but take all the input cuboids into account.

In [130]:
sealed class Cuboid(val width: LongRange, val height: LongRange, val depth: LongRange) {
    operator fun component1() = width; operator fun component2() = height; operator fun component3() = depth;
    class On(width: LongRange, height: LongRange, depth: LongRange) : Cuboid(width, height, depth)
    class Off(width: LongRange, height: LongRange, depth: LongRange) : Cuboid(width, height, depth)
}

fun <T : Cuboid> T.constructor(width: LongRange, height: LongRange, depth: LongRange): T = when (this as Cuboid) {
    is Cuboid.On -> Cuboid.On(width, height, depth) as T
    is Cuboid.Off -> Cuboid.Off(width, height, depth) as T
}

In [131]:
val cuboids: List<Cuboid> = File("Day22.input.txt")
    .bufferedReader()
    .readLines()
    .map { regex.find(it)!!.destructured }
    .map { (on, x1, x2, y1, y2, z1, z2) ->
        on to listOf(x1.toLong()..x2.toLong(), y1.toLong()..y2.toLong(), z1.toLong()..z2.toLong())
    }
    .map { (on, dimension) ->
        if (on == "on") Cuboid.On(dimension[0], dimension[1], dimension[2]) 
        else Cuboid.Off(dimension[0], dimension[1], dimension[2])
    }

In [132]:
// this: [], other: <>
infix fun LongRange.intersect(other: LongRange): LongRange? =
    // [<  >]   ||   <  [  ]  >   ||   <  [  >]   ||   [<  ]  >
    if (start >= other.start && endInclusive <= other.endInclusive)
        this
    // [  <  >  ]   ||   [  <  >]   ||   [<  >  ]
    else if (start <= other.start && endInclusive >= other.endInclusive)
        other
    // <  [  ]  >   ||   <  [>  ]
    else if (start > other.start && endInclusive > other.endInclusive && start <= other.endInclusive)
        start..other.endInclusive
    // [  <  ]  >   ||   [  <]  >
    else if (start < other.start && endInclusive < other.endInclusive && endInclusive >= other.start)
        other.start..endInclusive
    // [  ]  <  >   ||   <  > [  ]
    else null

infix fun LongRange.subtract(other: LongRange): List<LongRange> = (this intersect other)?.let {
    buildList {
        if (start < it.start) add(start..(it.start - 1))
        if (endInclusive > it.endInclusive) add((it.endInclusive + 1)..endInclusive)
    }
} ?: listOf(this)

infix fun <T : Cuboid> T.intersect(other: Cuboid): T? = (width intersect other.width)?.let { w ->
    (height intersect other.height)?.let { h ->
        (depth intersect other.depth)?.let { d ->
            constructor(w, h, d)
        }
    }
}

infix fun <T : Cuboid> T.subtract(other: Cuboid): List<T> = (this intersect other)?.let { (w, h, d) ->
    buildList {
        (width subtract w).map { constructor(it, height, depth) }.onEach(this::add)
        (height subtract h).map { constructor(w, it, depth) }.onEach(this::add)
        (depth subtract d).map { constructor(w, h, it) }.onEach(this::add)
    }
} ?: listOf(this)

fun List<Cuboid>.reboot(): List<Cuboid.On> = fold(mutableListOf<Cuboid.On>()) { acc, cuboid ->
    fun off(): MutableList<Cuboid.On> = acc.flatMap { it subtract cuboid }.toMutableList()
    when (cuboid) {
        is Cuboid.On -> off().apply { add(cuboid) }
        is Cuboid.Off -> off()
    }
}

fun LongRange.length() = endInclusive - start + 1

fun Cuboid.volume(): Long = width.length() * height.length() * depth.length()

cuboids.reboot().sumOf { it.volume() }

1211172281877240

### Notes

This problem definitely took a bit but needed to be continuously broken down for me to make sense of it. I first tried to think of what it means to intersect a square and recognized that I could describe the intersection as some combination of the 4 intersecting sides. With this in mind, I then realized that that we can then consider what the intersection on a line looked like. This lead us to drawing and writing out `LongRange::intersect`.

Learned that taking the `intersect` of two `...Range` produces a `LinkedHashSet` of all the elements (`IntRange.intersect(IntRange) -> Set`), rather than another range, so unfortunately we have to write our own. The same applies for `LongRange::subtract`.

We'll need to find out if any two `Cuboid`s intersect so we can properly turn cubes off. Here we can just check if all of their lines will intersect and the resulting line segments will form a new intersection cuboid. To subtract one cuboid from another, we can take that intersection previously computed and iterate through each dimension (easier to see in 2D).

After that the last "gotcha" is that when adding cuboids to the "all on" list, we can have overlaps. So we first have to `Cuboid::subtract` all the `Cuboid.On`s to make sure the new one we're adding does not overlap.

Lastly, once we've made sure none of the cuboids overlap, the number of "on"s is just the volume of each cube.

Overall, pretty happy with how clean the solution turned out despite it taking awhile for the problem to click.